In [69]:
import os
import numpy as np
from datetime import datetime
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from study_common import *
from study_notebook_helper import *
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
import itertools
import random

d = len(TAXONOMY)
study_datasets = [name for name in os.listdir(STUDY_RESULTS_PATH) if name.startswith("dataset-")]
@interact(dataset=study_datasets)
def main(dataset=study_datasets[0]):
    dataset_name = dataset[len("dataset-"):-len(".pickle")]
    study: STUDY_TYPE = None
    with open(STUDY_RESULTS_PATH + dataset, "rb") as data_file:
        study = pickle.load(data_file)
    print("This is a study on " + study[0])
    random.Random("erik" + dataset).shuffle(study[1])  # TODO NO!
    correct_labels = [entry[0] for entry in study[1]]
    
    print()

    result_vote_data = [name for name in os.listdir(STUDY_RESULTS_PATH) if name.startswith("votes-") and dataset_name in name]
    if len(result_vote_data) == 0:
        print("Oh no, there is no data yet! Please go and do the study!")
        return
    print("Chose votes to anaylze for " + dataset_name)
    @interact(result_data=result_vote_data)
    def analyze(result_data=result_vote_data[0]):
        votes = None
        with open(STUDY_RESULTS_PATH + result_data, "rb") as votes_file:
            votes = pickle.load(votes_file)
        display_labels = [t[1] for t in TAXONOMY]
        
        if True:  # reorder the matrix columns to have everything as close as possible to the middle!
            confusion = [[0] * d for t in range(d)]
            for correct, pred in zip(correct_labels, votes):
                confusion[correct][pred] += 1
            print(np.array(confusion))
            key=lambda order: sum((confusion[order[i1]][order[i2]] + confusion[order[i2]][order[i1]]) * (abs(i1 - i2) ** 2) for i1, i2 in itertools.combinations(range(d), 2))
            best_ordering = min(
                itertools.permutations(range(d), d),
                key=key
            )
            # best_ordering = random.choice(list(itertools.permutations(range(d), d)))  # use a random one!
            print(best_ordering)
            best_ordering_inv = [0 for i in range(d)]
            for i, val in enumerate(best_ordering):
                best_ordering_inv[val] = i
            print(best_ordering_inv)

            display_labels = [TAXONOMY[best_ordering[ti]][1] for ti in range(len(TAXONOMY))]
            for i in range(len(votes)):
                votes[i] = best_ordering_inv[votes[i]]
                correct_labels[i] = best_ordering_inv[correct_labels[i]]
            
        print(correct_labels)
        print(votes)
        
        # https://github.com/scikit-learn/scikit-learn/blob/cb7271339e56631fe47a22e259c98716f14f6894/sklearn/metrics/_plot/confusion_matrix.py#L566
        cm = confusion_matrix(correct_labels, votes, normalize=None)  # normalize="true"
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
        matrix_plot = disp.plot(
            include_values=True,
            cmap='viridis',
            ax=None,
            xticks_rotation='vertical',# 'horizontal',
            values_format=None,
            colorbar=True,
        )
        # https://stackoverflow.com/a/43153984/4354423
        plt.setp(matrix_plot.ax_.get_xticklabels(), rotation=30, ha="right")
        

interactive(children=(Dropdown(description='dataset', options=('dataset-junit-team_junit4.pickle', 'dataset-jf…